# Task4

Index_X = FSR_for_force, FSR_for_coord

Index_y = force, x_coord, y_coord

Data = Splited by Subject

## Run result

https://wandb.ai/seokjin/FSR-prediction/groups/FSR_Trainable_2023-08-07_11-13-30/workspace?workspace=user-seokjin

## Experiment id

FSR_Trainable_2023-08-07_11-13-30

## Best metric (RMSE)

103.112

In [1]:
def define_searchspace(trial):
    model = trial.suggest_categorical('model', ['fsr_model.CNN_LSTM'])
    if model == 'fsr_model.LSTM':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    elif model == 'fsr_model.CNN_LSTM':
        trial.suggest_categorical('model_args/cnn_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_categorical('model_args/lstm_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/cnn_num_layer', 1, 8)
        trial.suggest_int('model_args/lstm_num_layer', 1, 8)
    elif model == 'fsr_model.ANN':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    trial.suggest_categorical('criterion', ['torch.nn.MSELoss'])
    trial.suggest_categorical('optimizer', [
        'torch.optim.Adam',
        'torch.optim.NAdam',
        'torch.optim.Adagrad',
        'torch.optim.RAdam',
        'torch.optim.SGD',
    ])
    trial.suggest_float('optimizer_args/lr', 1e-5, 1e-1, log=True)
    imputer = trial.suggest_categorical('imputer', ['sklearn.impute.SimpleImputer'])
    if imputer == 'sklearn.impute.SimpleImputer':
        trial.suggest_categorical('imputer_args/strategy', [
            'mean',
            'median',
        ])
    trial.suggest_categorical('scaler', [ 
        'sklearn.preprocessing.StandardScaler',
        'sklearn.preprocessing.MinMaxScaler',
        'sklearn.preprocessing.RobustScaler',
    ])
    return {
        'index_X': ['FSR_for_force', 'FSR_for_coord'],
        'index_y': ['force', 'x_coord', 'y_coord'],
        'data_loader': 'fsr_data.get_index_splited_by_subject'
    }

In [2]:
import ray.tune
import ray.air
import ray.air.integrations.wandb
import ray.tune.schedulers
from fsr_trainable import FSR_Trainable
import ray.tune.search
import ray.tune.search.optuna

tuner = ray.tune.Tuner(
    trainable=ray.tune.with_resources(
        FSR_Trainable, {'cpu':2},
    ),
    tune_config=ray.tune.TuneConfig(
        num_samples=100,
        scheduler=ray.tune.schedulers.ASHAScheduler(
            max_t=100,
            grace_period=1,
            reduction_factor=2,
            brackets=1,
            metric='rmse',
            mode='min',
        ),
        search_alg=ray.tune.search.optuna.OptunaSearch(
            space=define_searchspace,
            metric='rmse',
            mode='min',
        ),
    ), 
    run_config=ray.air.RunConfig(
        callbacks=[
            ray.air.integrations.wandb.WandbLoggerCallback(project='FSR-prediction'),
        ],
        checkpoint_config=ray.air.CheckpointConfig(
            num_to_keep=3,
            checkpoint_score_attribute='rmse',
            checkpoint_score_order='min',
            checkpoint_frequency=5,
            checkpoint_at_end=True,
        ),
    ), 
)

[I 2023-08-07 11:13:30,239] A new study created in memory with name: optuna


In [3]:
results = tuner.fit()

2023-08-07 11:13:32,417	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-08-07 11:13:33,650	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


2023-08-07 11:13:33,698	INFO wandb.py:320 -- Already logged into W&B.


Trial name,date,done,hostname,iterations_since_restore,mae,mape,node_ip,pid,rmse,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
FSR_Trainable_00767336,2023-08-07_11-23-21,True,DESKTOP-0P789CI,1,66.3592,2.80543e+15,172.26.215.93,154233,228.909,4.3516,4.3516,4.3516,1691375001,1,00767336
FSR_Trainable_00a89f50,2023-08-07_11-17-54,True,DESKTOP-0P789CI,4,67.0777,1.55196e+07,172.26.215.93,147116,216.38,19.4953,5.49213,19.4953,1691374674,4,00a89f50
FSR_Trainable_02093e1c,2023-08-07_11-20-59,True,DESKTOP-0P789CI,4,85.5653,7.06408e+07,172.26.215.93,151099,233.885,6.57998,1.63472,6.57998,1691374859,4,02093e1c
FSR_Trainable_04941f48,2023-08-07_11-34-22,True,DESKTOP-0P789CI,100,38.5098,1.52176e+07,172.26.215.93,164099,126.337,84.0677,0.675504,84.0677,1691375662,100,04941f48
FSR_Trainable_0620d1d0,2023-08-07_11-22-18,True,DESKTOP-0P789CI,8,66.733,2.25445e+07,172.26.215.93,152675,207.628,14.9542,1.87165,14.9542,1691374938,8,0620d1d0
FSR_Trainable_0b22ee3e,2023-08-07_11-29-45,True,DESKTOP-0P789CI,2,81.6878,3.62716e+07,172.26.215.93,160198,231.598,1.93238,0.883483,1.93238,1691375385,2,0b22ee3e
FSR_Trainable_10d594f8,2023-08-07_11-21-14,True,DESKTOP-0P789CI,1,96.1698,2.03767e+17,172.26.215.93,151564,281.002,1.5166,1.5166,1.5166,1691374874,1,10d594f8
FSR_Trainable_1223a976,2023-08-07_11-35-28,True,DESKTOP-0P789CI,100,37.097,1.44455e+07,172.26.215.93,164627,117.592,67.279,0.503871,67.279,1691375728,100,1223a976
FSR_Trainable_147eba8a,2023-08-07_11-19-08,True,DESKTOP-0P789CI,4,65.1665,1.00398e+07,172.26.215.93,148421,226.448,15.4134,3.65428,15.4134,1691374748,4,147eba8a
FSR_Trainable_1590aeaf,2023-08-07_11-29-51,True,DESKTOP-0P789CI,64,45.1956,1.62103e+07,172.26.215.93,159254,140.014,54.3161,0.699773,54.3161,1691375391,64,1590aeaf


(_WandbLoggingActor pid=142840) wandb: Currently logged in as: seokjin. Use `wandb login --relogin` to force relogin
(_WandbLoggingActor pid=142840) wandb: wandb version 0.15.8 is available!  To upgrade, please run:
(_WandbLoggingActor pid=142840) wandb:  $ pip install wandb --upgrade
(_WandbLoggingActor pid=142840) wandb: Tracking run with wandb version 0.15.4
(_WandbLoggingActor pid=142840) wandb: Run data is saved locally in /home/seokj/ray_results/FSR_Trainable_2023-08-07_11-13-30/FSR_Trainable_8b662eb9_1_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_subject,imputer=sklearn_impute_Simpl_2023-08-07_11-13-33/wandb/run-20230807_111344-8b662eb9
(_WandbLoggingActor pid=142840) wandb: Run `wandb offline` to turn off syncing.
(_WandbLoggingActor pid=142840) wandb: Syncing run FSR_Trainable_8b662eb9
(_WandbLoggingActor pid=142840) wandb: ⭐️ View project at https://wandb.ai/seokjin/FSR-prediction
(_WandbLoggingActor pid=142840) wandb: 🚀 View run at https://wandb.ai/se